<a href="https://colab.research.google.com/github/ToniRV/MIT_6.862_Applied_Machine_Learning/blob/master/MIT_6_036_Lab11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MIT 6.036 Spring 2020: Lab 11**

This colab notebook provides code and a framework for sections 2 and 3 of the lab.

## **Setup**

First, download the code distribution for this homework that contains test cases and helper functions.

Run the next code block to download and import the code for this lab.

In [2]:
!rm -rf code_for_lab11*
!wget --quiet https://introml.odl.mit.edu/cat-soop/_static/6.036/labs/lab11/code_for_lab_11.zip
!unzip code_for_lab_11.zip
!mv ./code_for_lab11/* ./

Archive:  code_for_lab_11.zip
   creating: code_for_lab11/
  inflating: code_for_lab11/code_for_lab11.py  
  inflating: code_for_lab11/metal_bands.txt  
  inflating: code_for_lab11/.DS_Store  
  inflating: code_for_lab11/util.py  
  inflating: code_for_lab11/companies.csv  
   creating: code_for_lab11/models/
  inflating: code_for_lab11/models/metal_rnn.p  
  inflating: code_for_lab11/models/MIT_classes_rnn.p  
  inflating: code_for_lab11/models/food_rnn.p  
  inflating: code_for_lab11/models/companies_rnn.p  
  inflating: code_for_lab11/sm.py    
  inflating: code_for_lab11/baskervilles.txt  
  inflating: code_for_lab11/MIT_classes.txt  
  inflating: code_for_lab11/basicEnglish.txt  
  inflating: code_for_lab11/simple_poem.txt  
  inflating: code_for_lab11/food.txt  
  inflating: code_for_lab11/mousquetaires.txt  
  inflating: code_for_lab11/companies.txt  
   creating: __MACOSX/
  inflating: __MACOSX/._code_for_lab11  


In [0]:
from sm import *
from util import *
from code_for_lab11 import *
import numpy as np
import _pickle as cPickle
m = cPickle.load(open('models/food_rnn.p', 'rb'))

## **Section 2, Problem B**

Below, you will find definitions of a procedure called *test_linear_accumulator* for training an rnn with input and output sequences of the kind produced by Accumulator. (Alternatively, you can look at *code_for_lab11.py* of the code file available for download) Study this function, in particular, the definition of the RNN instance; compare to your choices in the previous question.

In [0]:
# please evaluate this cell to see its output

def test_linear_accumulator(num_steps = 10000,
                            num_seqs = 400, seq_length = 40,
                            step_size = .01):
    # generate random training data: num_seqs of seq_length of random
    # numbers between -0.5 and 0.5.
    data = []
    for _ in range(num_seqs):           
        x = np.random.random((1, seq_length)) - 0.5 # seq in
        y = np.zeros((1, seq_length))               # seq out
        for j in range(seq_length):
            y[0, j] = x[0, j] + (0.0 if j == 0 else y[0, j-1])
        data.append((x, y))
    # specify rnn
    rnn = RNN(1, 1, 1, quadratic_loss, lambda z: z, quadratic_linear_gradient,
              step_size, lambda z: z, lambda z: 1)
    # train it
    rnn.train_seq_to_seq(data, num_steps)
    # print weights
    print("Wsx: ", rnn.Wsx); print("Wss: ", rnn.Wss); print("Wo: ", rnn.Wo); print("Wss0: ", rnn.Wss0); print("Wo0: ", rnn.Wo0)
    return rnn


Now run this function a few times, making sure that the training error is low and look at the final weights. Relate to the weights you chose above. Explain.

In [5]:
test_linear_accumulator()

1/10: training error 0.10965130122299203
2/10: training error 0.08017097075653189
3/10: training error 2.386918447160652e-05
4/10: training error 0.004187864759620643
5/10: training error 0.0014861850326898364
6/10: training error 0.07683477789102473
7/10: training error 1.2740239379311168e-11
8/10: training error 2.0362360289305708e-16
9/10: training error 0.04580439389405539
Wsx:  [[2.42525127]]
Wss:  [[0.99619299]]
Wo:  [[0.40954527]]
Wss0:  [[0.00046585]]
Wo0:  [[-0.00218033]]


## **Section 3**

### Generating sequences

We are interested in building an RNN model that can predict the next
element in a sequence (this is sometimes referred to as a "language"
model).    The particular form of the model that we will look at is:

<displaymath>
\begin{align}
x_t &= \phi(c_{t-1})\\
s_t &= \tanh(W^{ss} s_{t-1} + W^{sx} x_t + W^{ss}_0)\\
p_t &= \text{softmax}(W^o s_t + W^o_0)
\end{align}
</displaymath>

**Please refer to the lab's prompt for more definitions and details about this section.**

As we just said, you can find more details in the lab's prompt, but here some extra details:

Training is as follows:
<ul>
<li> For each sequence in the input data, it feeds in character <math>t-1</math> from
  the training data and predicts character $t$.  </li>
</ul>

Generation is as follows:
<ul>
<li> Starting with the `start` symbol ('.'), it predicts a next
  character based on the softmax distribution in the trained model,
  then it feeds that character into the model and repeats until an
  <i>end</i> symbol ('\n') is generated.</li>
</ul>




For each of 

*   "aaaaaaaaaa"
*   "aabaaabbaaaababaabaa"
*   "abcdefghijklmnopqrstuvwxyz"
*   "abcabcabcabcabc"

train an RNN and assess the difficulty of learning each string. You may control *num_hidden* and *num_steps* to facilitate your training in the code below.

**Note**: The output of `test_word` is 100 sequences produced by running the generation process on a trained model.


In [6]:
test_word(word="aaaaaaaaaa", interactive = False, num_hidden=1, num_steps=10000, step_size=0.005)

1/10: training error 0.3058238806070037
2/10: training error 0.03325205127086638
3/10: training error 0.008949570684933563
4/10: training error 0.0038594599359628914
5/10: training error 0.0018758980894445115
6/10: training error 0.04337550443789023
7/10: training error 0.002063590198645276
8/10: training error 0.0013923343442130978
9/10: training error 0.0009078512621424098
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n'
'.aaaaaaaaaa\n

(<code_for_lab11.RNN at 0x7f57db50bb38>,
 <code_for_lab11.OneHotCodec at 0x7f57db50bac8>)

In [7]:
test_word(word="aabaaabbaaaababaabaa", interactive = False, num_hidden=5, num_steps=20000, step_size=0.005)

1/10: training error 0.3562842395495819
2/10: training error 0.17663127472291945
3/10: training error 0.05366542651599676
4/10: training error 0.0040842361912864595
5/10: training error 0.0012079256798835163
6/10: training error 0.0003862590087258371
7/10: training error 0.0001259288928377182
8/10: training error 4.149626661477658e-05
9/10: training error 1.3794086659829036e-05
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaabaa\n'
'.aabaaabbaaaababaaba

(<code_for_lab11.RNN at 0x7f57db503d30>,
 <code_for_lab11.OneHotCodec at 0x7f57db503dd8>)

In [8]:
test_word(word="abcdefghijklmnopqrstuvwxyz", interactive = False, num_hidden=28, num_steps=100, step_size=0.005)

1/10: training error 3.611447829999854
2/10: training error 3.135614837216415
3/10: training error 2.5192144928880924
4/10: training error 1.5501658742339859
5/10: training error 0.7822380661404886
6/10: training error 0.3524361121429647
7/10: training error 0.1616303609148579
8/10: training error 0.08686653836513578
9/10: training error 0.05582428347011883
'.abcdefghijklmnosqrstuvwxyz\n'
'.abcdbfghijklmnopqrstuvwxyz\n'
'.abcdifghijklmnopqrstuvwxyc\n'
'.abcdefghijkomnopqrstuvwxyz\n'
'.abcdefghijklmnopqrstuvwxyz\n'
'.abcdefgyijklmnopqrstuvwxyz\n'
'.abtdgfghijklmnopqrvtuvwxyz\n'
'.abcuefghijklmnopqrstuvwxyz\n'
'.abcdefghijklmqqpqrstuvwxyz\n'
'.abcdefghijklmnopqrstuvwxyz\n'
'.abbretuvwl\n'
'.abcdefghijklmnopqrstuvwxyz\n'
'.abcdefghijklmnopqrstuvaxyz\n'
'.abcdefghijkhmnopqrstuvwxyz\n'
'.abcdefghijklmnopqrstuvwxyz\n'
'.abcdefghijklmnopqrgtuvwxyz\n'
'.abcdefghijklmnopqrstuvwxyz\n'
'.abcdefghijklmnopqrbcuvwxyz\n'
'.fbcdefghijklmnopqrstuvwxyz\n'
'.abcdefghijklmnopqrstuvwxyz\n'
'.abcdefghijklmn

(<code_for_lab11.RNN at 0x7f57db5038d0>,
 <code_for_lab11.OneHotCodec at 0x7f57db503c88>)

In [9]:
test_word(word="abcabcabcabcabc", interactive = False, num_hidden=1, num_steps=10000, step_size=0.005)

1/10: training error 0.6241473815778071
2/10: training error 0.27025810189508526
3/10: training error 0.20989295950357387
4/10: training error 0.18928764357001288
5/10: training error 0.18008102213384938
6/10: training error 0.17536219198461328
7/10: training error 0.17275233920239447
8/10: training error 0.17124376356099869
9/10: training error 0.17034797918060607
'.abc\n'
'.abc\n'
'.abcabcabcabcabc\n'
'.abcabcabcabc\n'
'.abcabcabcabcabc\n'
'.abc\n'
'.abcabcabcabcabcabcabc\n'
'.abcabcabcabcabcabc\n'
'.abc\n'
'.abcabcabcabcabc\n'
'.abcabc\n'
'.abcabcabcabcabcabc\n'
'.abc\n'
'.abcabcabc\n'
'.abcabc\n'
'.abcabcabcabc\n'
'.abc\n'
'.abcabc\n'
'.abcabcabcabcabcabcabcabcabcabcbcabcabcabc\n'
'.abc\n'
'.abc\n'
'.abcabc\n'
'.abcabcabc\n'
'.abcabcabcabcabcabcabc\n'
'.abcabcabcabcabcabc\n'
'.abcabcabcabcabcabc\n'
'.abcabcabc\n'
'.abcabcabcabcabcabc\n'
'.abcabcabcabcabc\n'
'.abcabcabcabcabcabcabcabc\n'
'.abcabcabcabcabcabcabc\n'
'.abcabc\n'
'.abcabcabcabcabc\n'
'.abcabcabcabcabcabcabcabcabcabc\n'


(<code_for_lab11.RNN at 0x7f57db518240>,
 <code_for_lab11.OneHotCodec at 0x7f57db503668>)

## Problem 3D

The *test_company_names* function uses a file <a href="https://docs.google.com/spreadsheets/d/16wbhwRwu5AP4gydX9buHZj884v48HURaalYtKmdT8r0/edit?usp=sharing">companies.txt</a> of company names for training and generates new names.

Experiment with different values of `num_steps`; more steps gives better results. Try it in interactive mode; it's more fun.

**In running `test_company_names`, first try with `train=True`; you can train the RNN yourself with this option. If this takes too much time, run  `test_company_names` with `train=False`; you can simply load a pre-trained model with this option.**

Please note that when we train our models, we need to decide what will be our vocabulary size (i.e. how many classes/characters/words our model will learn to predict). Hence, it is possible that when you input a character/word that is out of that vocabulary (depending on how the model is trained -- character-based or word-based), the model crashes (in more sophisticated solutions we could use a `<unknown>` token to represent **tokens out of vocabulary.**)

These models are trained by minimizing NLL.



In [0]:
train = False

test_company_names(interactive = True, interactive_top5 = False,
                   train=train)

#Trained with num_hidden = 150, num_steps = 20000,
#             step_size = .001, split=0


Type in a start-of-sequence string
(the model will try to generate the rest): appl
'.applers\n'


In [0]:
# This cell may take a long time because here we are training a model!
train = True

test_company_names(interactive = False, interactive_top5 = False, 
           split=0, num_hidden = 150, 
           num_steps = 20000, step_size = .001, train=train)

## Problem 3E

The *test_food* function uses a file <a href="https://docs.google.com/spreadsheets/d/12eomCfekSdqTlOB4Xwp3n7_FN0SVdVrTDLVUBq-1jyw/edit?usp=sharing">food.txt</a> of recipe names. Experiment with different values of num_steps; more steps gives better results. Try it in interactive mode; it's more fun. Note the difference between starting with a capital letter versus lower case letter.

**In running `test_food`, first try with `train=True`; you can train the RNN yourself with this option. If this takes too much time, run  `test_food` with `train=False`; you can simply load a pre-trained model with this option.**



In [0]:
train = False

test_food(interactive = True, interactive_top5 = False, 
          train=train)

#Trained with num_hidden = 150, num_steps = 20000,
#             step_size = .001, split=0

In [0]:
# This cell may take a long time because here we are training a model!
train = True

test_food(interactive = True, interactive_top5 = False, 
          split=0, num_hidden = 150, 
          num_steps = 20000, step_size = .001, train=train)

## Problem 3F

Now run one of <code>test_food, test_company_names</code> with <code>interactive_top5=True</code>; play with them for a while. Can you notice the mechanism of how these functions generate their output? 

Use the cell below to run one of <code> test_food, test_company_names</code> with <code>interactive_top5=True</code>!

- What is the mechanism by which these RNNs generate their output? More specifically, what character does the trained RNN seem to output at each location?
- Also, what would be a reasonable criterion for the above-trained RNNs to stop generating more characters? When does it seem to stop? Be prepared to explain this during checkoff.

In [0]:
train = True

test_food(interactive = True, interactive_top5 = True,
          train=train)

# Pre trained with split=0, num_hidden = 150,
#                   num_steps = 20000, step_size = .001, 

In [0]:
train = False

test_company_names(interactive = True, interactive_top5 = True,
                  train=train)

# Pre trained with split=0, num_hidden = 150,
#                   num_steps = 20000, step_size = .001, 